In [1]:
ref = '/mnt/d/osm2lulc02/lmt/rst_terceira.tif'

osm_db = 'osm_terceira'

nomenclature = 'clc'

ws = '/mnt/d/osm2lulc02/debug'

In [2]:
import os
import datetime as dt

from glass.ete.osm2lulc.utilsv15 import nomenclature_id, osmfeat_by_rule
from glass.pys.oss import fprop
from glass.cons.osmtolulc import OSM2LULC_DB, DB_SCHEMA, OSM_TABLES
from glass.sql.q import q_to_obj
from glass.sql.q import q_to_ntbl
from glass.prop.prj import get_epsg

from glass.wenv.grs import run_grass
from glass.ete.osm2lulc.mod1 import grs_vector
from glass.ete.osm2lulc.mod2 import vec_roads

from glass.gp.gen.sql import st_dissolve

In [3]:
epsg = get_epsg(ref)

In [3]:
from glass.ete.otol.tools import module_osmtags

tags = module_osmtags(2)

In [6]:
tags[tags.name == 'roads'].lulc_id.unique()

array([86])

In [3]:
tags.name.unique()

array(['selection', 'buildings', 'area_upper', 'basic_buffer', 'roads',
       'area_lower'], dtype=object)

In [5]:
tags[
    (tags['value'] == 'yes') &
    (tags['key'] == 'building')
]

,key,value,name,lulc_id,buffer_dist,area
12,building,yes,buildings,28,NaN,NaN


In [1]:
from glass.ete.otol.pg import osm_to_lulc
from glass.prop.prj import get_epsg
import datetime
import os

osm = '/mnt/d/osm2lulc02/osmdata/osm_terceira.xml'

nomenclature = 'clc'

refraster = '/mnt/d/osm2lulc02/lmt/rst_terceira.tif'

result = '/mnt/d/osm2lulc02/debug/terceira.gpkg'

db = '/mnt/d/osm2lulc02/debug/db_terceira_jun.sql'

time_a = datetime.datetime.now().replace(microsecond=0)
res, logs = osm_to_lulc(
    osm, nomenclature, get_epsg(refraster),
    result, savedb=db, tmpfld=os.path.dirname(result)
)
time_b = datetime.datetime.now().replace(microsecond=0)

print(time_b - time_a)

/home/jasp/.virtualenvs/genv/lib/python3.8/site-packages/geopandas/_compat.py:112: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(


0:00:34


In [1]:
gpkg = '/mnt/d/osm2lulc02/debug/modules_res.gpkg'
res = '/mnt/d/osm2lulc02/debug/modules_merge.gpkg'
olayer = 'lulc_v1'

from glass.pys import execmd
from glass.prop import drv_name
from glass.prop.sql import lst_tbl
import geopandas as gpd
import fiona

layers = fiona.listlayers(gpkg)

drv = drv_name(res)

cmdo = execmd(f'ogr2ogr -f "{drv}" {res} -nln {olayer} {gpkg} {layers[0]}')

lcmd = [execmd(
    f'ogr2ogr -f "{drv}" -update -append {res} -nln {olayer} {gpkg} {layers[i]}'
) for i in range(1, len(layers))]

/home/jasp/.virtualenvs/genv/lib/python3.8/site-packages/geopandas/_compat.py:112: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(


In [4]:
layers

['module_1', 'module_2', 'module_3', 'module_4', 'module_5', 'module_6']

In [ ]:
from glass.ete.osm2lulc.grs import osm_to_lulc
from glass.ete.osm2lulc.utils import record_time_consumed
import datetime
import os
from glass.pys.oss import fprop, mkdir
from glass.pys.tm import now_as_str

In [ ]:
time_a = datetime.datetime.now().replace(microsecond=0)

ws = mkdir(os.path.join(
    os.path.dirname(osm),
    fprop(result, 'fn')
), overwrite=True)

shp, time_check = osm_to_lulc(
    osm, nomenclature, refraster,
    result, tmpfld=ws,
    overwrite=True, savedb=db
)

time_b = datetime.datetime.now().replace(microsecond=0)
    
# Record time consumed in xlsx table
record_time_consumed(time_check, os.path.join(
    os.path.dirname(result),
    fprop(result, 'fn') + '.xlsx'
))

print(time_b - time_a)